###  Questions

1. How does total revenue change by month?
   - Metric: total revenue (sum of amount)
   - Grouped by: month
   - Output: monthly trend table and line chart

2. How does total revenue and number of orders differ by country?
   - Metric: total revenue, order count
   - Grouped by: country
   - Output: summary table and bar chart

3. What is the distribution of order amount after winsorization?
   - Metric: amount_winsor
   - Output: histogram
   - Goal: understand spread and skewness



From processed analytics table 

1. Setup & Imports


In [18]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px


ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA = ROOT / "data" / "processed" / "analytics_table.parquet"
FIGS = ROOT / "reports" / "figures"
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, scale: int = 2):
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

2. Load Processed Data


In [19]:
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
data_path = ROOT / "data" / "processed" / "analytics_table.parquet"

df = pd.read_parquet(data_path)
df.head()


,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor
0,ORD-01,U101,50.0,1,2025-03-01 09:00:00+00:00,paid,paid,False,False,2025-03-01,2025,2025-03,Saturday,9,SA,2025-02-01,50.0
1,ORD-02,U102,20.0,2,2025-03-01 10:30:00+00:00,PAID,paid,False,False,2025-03-01,2025,2025-03,Saturday,10,EG,2025-02-05,20.3
2,ORD-03,U103,30.0,1,2025-03-02 11:00:00+00:00,refund,refund,False,False,2025-03-02,2025,2025-03,Sunday,11,SA,2025-02-10,30.0
3,ORD-04,U101,75.5,<NA>,2025-03-03 14:00:00+00:00,paid,paid,False,True,2025-03-03,2025,2025-03,Monday,14,SA,2025-02-01,74.735
4,ORD-05,U104,<NA>,1,2025-03-04 16:30:00+00:00,paid,paid,True,False,2025-03-04,2025,2025-03,Tuesday,16,KW,2025-02-15,<NA>


3. Quick Audit


In [20]:
print("rows:", len(df), "cols:", len(df.columns))

df.dtypes.head(15)

missing = df.isna().sum().sort_values(ascending=False).head(10)
missing


rows: 5 cols: 17


amount            1
amount_winsor     1
quantity          1
order_id          0
user_id           0
status            0
status_clean      0
amount__isna      0
created_at        0
quantity__isna    0
dtype: int64

summary:
1- Dataset contains 5 rows and 17 columns
2- Missing values in (amount, quantity, amount_winsor)


In [21]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   order_id        5 non-null      string             
 1   user_id         5 non-null      string             
 2   amount          4 non-null      Float64            
 3   quantity        4 non-null      Int64              
 4   created_at      5 non-null      datetime64[ns, UTC]
 5   status          5 non-null      object             
 6   status_clean    5 non-null      object             
 7   amount__isna    5 non-null      bool               
 8   quantity__isna  5 non-null      bool               
 9   date            5 non-null      object             
 10  year            5 non-null      int32              
 11  month           5 non-null      string             
 12  dow             5 non-null      object             
 13  hour            5 non-null      int32  

In [22]:
df.isna().mean()


order_id          0.0
user_id           0.0
amount            0.2
quantity          0.2
created_at        0.0
status            0.0
status_clean      0.0
amount__isna      0.0
quantity__isna    0.0
date              0.0
year              0.0
month             0.0
dow               0.0
hour              0.0
country           0.0
signup_date       0.0
amount_winsor     0.2
dtype: float64

4. Questions + results

### Q1. Revenue by country

How does revenue performance differ across countries?


In [23]:
country_grp = df.groupby("country", dropna=False)

orders_count = country_grp["order_id"].count()
total_revenue = country_grp["amount"].sum()
avg_order_value = country_grp["amount"].mean()

rev = pd.DataFrame({
    "country": orders_count.index,
    "n": orders_count.values,
    "revenue": total_revenue.values,
    "aov": avg_order_value.values,
}).sort_values("revenue", ascending=False)

rev


,country,n,revenue,aov
2,SA,3,155.5,51.833333
0,EG,1,20.0,20.0
1,KW,1,0.0,<NA>


In [24]:
fig = px.bar(
    data_frame=rev,
    x="country",
    y="revenue",
    color="country"
)

fig.update_layout(
    title="Total Revenue by Country",
    xaxis_title="Country",
    yaxis_title="Total Revenue",
    showlegend=False
)

save_fig(fig, FIGS / "revenue_by_country.png")
fig


Interpretation:
1- السعوديه تحقق اعلى ايرادات مقارنة بباقي الدول
2- مصر لديها عدد طلبات اقل وايرادات اقل
3- الكويت لديها ايرادات محدوده بسبب وجود قيم مفقودة في عمود المبلغ

Caveat:
1- ممكن تكون مقارنة الايرادات غير دقيقة بشكل كامل بسبب وجود طلبات تحتوي على قيم مفقودة في عمود 
amount



Q2. Revenue trend (monthly)
  
How does total revenue change over time on a monthly basis?


In [25]:
trend = (
    df.groupby("month", dropna=False)
      .agg(
          n=("order_id", "size"),
          revenue=("amount", "sum")
      )
      .reset_index()
      .sort_values("month")
)

trend


,month,n,revenue
0,2025-03,5,175.5


In [26]:
fig = px.line(
    trend,
    x="month",
    y="revenue",
    title="Revenue over time (monthly)"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Revenue"
)

save_fig(fig, FIGS / "revenue_trend_monthly.png")
fig

Interpretation:
1- عدد الطلبات خلال هذاالشهر محدود، بالتالي  قيمه الايرادات الإجمالية تعكس نشاط زمني قصير 
2- مانقدر نستنتج نمط نمو  طويل المدى من البيانات الحالية

Caveat:
1- تحليل الاتجاه محدود بسبب توفر بيانات لشهر واحد فقط

Q3. Distribution of Order Amount (After Winsorization)

In [31]:
fig = px.histogram(
    df,
    x="amount_winsor",
    nbins=20,
    title="Distribution of Order Amount after Winsorization"
)

fig.update_layout(
    xaxis_title="Amount (winsorized)",
    yaxis_title="Count"
)

save_fig(fig, FIGS / "amount_distribution_winsor.png")
fig


(Interpretation)
- تقليل القيم المتطرفة ادى الى ان التوزيع اكثر استقرار وسهل في التفسير.

(Caveat)
- على الرغم من winsorization
بس التوزيع ممكن لا يعكس السلوك الحقيقي للقيم المتطرفة الأصلية


5. Bootstrap Comparison


In [27]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }


In [30]:
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("EG"), "is_refund"]

print("n_SA:", len(a), "n_EG:", len(b))

res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(res)


n_SA: 3 n_EG: 1
{'diff_mean': 0.3333333333333333, 'ci_low': 0.0, 'ci_high': 1.0}


 6. Findings & Caveats


 (Findings): 
- الايرادات تختلف بشكل واضح بين الدول، حيث تظهر بعض الدول مساهمة اعلى في اجمالي الايرادات
- تطبيق winsorization 
على قيمة الطلب ساعد في تقليل تاثير القيم المتطرفة وتوضيح التوزيع


(Caveats): 
- حجم البيانات صغير، مما يؤثر من دقه الاستنتاجات الاحصائيه 
- وجود قيم مفقودة في بعض الاعمده قد يؤثر على دقة النتائج 
- فترة الزمن قصيرة ولا تمثل سلوك طويل المدى 
